# Henter API

Her henter vi inn API-et og lagrer det i en csv fil
Vi henter data for lufttemperatur, nedmørsmengde og vindhastighet over 45 år i Oslo på værstasjonen Oslo-Blindern

In [9]:
# Importerer biblioteker
import requests
import pandas as pd

# Skriver inn client Id
client_id = '61f26ba4-3f68-4e56-a3b8-09733ceb82ed'

# Definerer endpoint og parametere
endpoint = 'https://frost.met.no/observations/v0.jsonld'
parameters = {
    'sources': 'SN18700',
    'elements': 'mean(air_temperature P1D),sum(precipitation_amount P1D),mean(wind_speed P1D)',
    'referencetime': '1975-01-01/2020-12-31',
}
# Sender request
r = requests.get(endpoint, parameters, auth=(client_id,''))
# Får inn json data
json = r.json()

# Sjekker requesten, om den er 200 er det godkjent og dataen er mottatt. Sjekker errorene hvis status_code ikke er 
if r.status_code == 200:
    data = json['data']
    print('Data retrieved from frost.met.no!')
else:
    print('Error! Returned status code %s' % r.status_code)
    print('Message: %s' % json['error']['message'])
    print('Reason: %s' % json['error']['reason'])


Data retrieved from frost.met.no!


# Vis data

Her viser vi at dataen eksisterer 

In [10]:
# Viser dataen i en dataframe som en tabell
df = pd.DataFrame()
for i in range(len(data)):
    row = pd.DataFrame(data[i]['observations'])
    row['referenceTime'] = data[i]['referenceTime']
    row['sourceId'] = data[i]['sourceId']
    df = pd.concat([df, row])

df = df.reset_index()

print(df)

df.to_csv('../data/data_45years.csv')

       index                      elementId  value  unit  \
0          0      mean(air_temperature P1D)   -2.3  degC   
1          1      mean(air_temperature P1D)   -0.4  degC   
2          2  sum(precipitation_amount P1D)    0.0    mm   
3          3  sum(precipitation_amount P1D)    7.4    mm   
4          4           mean(wind_speed P1D)    1.0   m/s   
...      ...                            ...    ...   ...   
83989      0      mean(air_temperature P1D)    1.7  degC   
83990      1      mean(air_temperature P1D)    2.3  degC   
83991      2  sum(precipitation_amount P1D)    5.9    mm   
83992      3  sum(precipitation_amount P1D)    5.6    mm   
83993      4           mean(wind_speed P1D)    5.1   m/s   

                                                   level timeOffset  \
0      {'levelType': 'height_above_ground', 'unit': '...       PT0H   
1      {'levelType': 'height_above_ground', 'unit': '...       PT6H   
2                                                    NaN      PT18

# Analyserer data

Vi ser at vi har mange unødvendige kolonner, som gjør dataen uoversiktlig. Derfor fjerner vi kolonnene: 
level, timeResolution, timeSeriesId, performanceCategory, exposureCategory, qualityCode, sourceId

Dette gjør vi med funksjonen drop(), der vi fjerner kolonnene vi ønsker

Deretter lagrer vi dette i en ny csv-fil som vi kaller data_45years_update1

In [11]:
# Fjerner unødvendige kolonner
df = df.drop(['level','timeResolution','timeSeriesId','performanceCategory','exposureCategory','qualityCode','sourceId'], axis=1)
print(df)


df.to_csv('../data/data_45years_update1.csv')



       index                      elementId  value  unit timeOffset  \
0          0      mean(air_temperature P1D)   -2.3  degC       PT0H   
1          1      mean(air_temperature P1D)   -0.4  degC       PT6H   
2          2  sum(precipitation_amount P1D)    0.0    mm      PT18H   
3          3  sum(precipitation_amount P1D)    7.4    mm       PT6H   
4          4           mean(wind_speed P1D)    1.0   m/s       PT0H   
...      ...                            ...    ...   ...        ...   
83989      0      mean(air_temperature P1D)    1.7  degC       PT0H   
83990      1      mean(air_temperature P1D)    2.3  degC       PT6H   
83991      2  sum(precipitation_amount P1D)    5.9    mm      PT18H   
83992      3  sum(precipitation_amount P1D)    5.6    mm       PT6H   
83993      4           mean(wind_speed P1D)    5.1   m/s       PT0H   

                  referenceTime  
0      1975-01-01T00:00:00.000Z  
1      1975-01-01T00:00:00.000Z  
2      1975-01-01T00:00:00.000Z  
3      1975

# Legger til en kolonne 'dato'

Vi legger til en kolonne som viser dato, uten klokkeslett

Dette gjør vi ved å legge til kolonnen vi kaller "date" og gjør om teksten i kolonnen "referenceTime" til datetime-format, slik at vi kan hente ut datoen uten tiden. 

In [12]:
#oppretter en ny kolone med dato uten tiden

#Gjør om refrenceTime til datetime_format og henter ut datoen, og kaller denne kolonnen for "date"
df['date'] = pd.to_datetime(df['referenceTime']).dt.date

#Legger den nye kolonnen "date" først, og de gamle kolonnene etter den, også oppdaterer dataframen med nye kolonner
cols = ['date'] + [col for col in df.columns if col != 'date']
df = df[cols]

df.to_csv('../data/test_date.csv')

# Fjerner verdier som ikke gir mening
Vi velger å fjerne rader for temperatur der verdiene enten er under -50 eller over 60, fordi dette gir ikke mening og er sannsynligvis feil måling

In [13]:
# Sjekker antall rader før filtrering
rows_before = len(df)

# Bruk df.loc for å kun beholde verdier på målinger som ikke gir mening, basert på historisk data for våre elementer i Norge
df_filtered = df.loc[((df['elementId'] == 'mean(air_temperature P1D)') & 
                     (df['value'] >= -50) & 
                     (df['value'] <= 45)) |

                     ((df['elementId'] == 'sum(precipitation_amount P1D)') & 
                     (df['value'] >= 0) & 
                     (df['value'] <= 230)) |

                     ((df['elementId'] == 'mean(wind_speed P1D)') & 
                     (df['value'] >= 0) & 
                     (df['value'] <= 82)) ]

# Sjekker antall rader etter filtrering
rows_after = len(df_filtered)

# Sjekker antall rader fjernet
rows_removed = rows_before - rows_after

print(f'Antall rader fjernet er: {rows_removed}')

# Skriv ut den filtrerte DataFrame-en
print(df_filtered)

# Lagrer dataen som er innenfor gyldighetsområde i en csv-fil
df_filtered.to_csv('../data/valid_data.csv')

Antall rader fjernet er: 0
             date  index                      elementId  value  unit  \
0      1975-01-01      0      mean(air_temperature P1D)   -2.3  degC   
1      1975-01-01      1      mean(air_temperature P1D)   -0.4  degC   
2      1975-01-01      2  sum(precipitation_amount P1D)    0.0    mm   
3      1975-01-01      3  sum(precipitation_amount P1D)    7.4    mm   
4      1975-01-01      4           mean(wind_speed P1D)    1.0   m/s   
...           ...    ...                            ...    ...   ...   
83989  2020-12-30      0      mean(air_temperature P1D)    1.7  degC   
83990  2020-12-30      1      mean(air_temperature P1D)    2.3  degC   
83991  2020-12-30      2  sum(precipitation_amount P1D)    5.9    mm   
83992  2020-12-30      3  sum(precipitation_amount P1D)    5.6    mm   
83993  2020-12-30      4           mean(wind_speed P1D)    5.1   m/s   

      timeOffset             referenceTime  
0           PT0H  1975-01-01T00:00:00.000Z  
1           PT6H  

In [14]:
# Sjekker om det er noen manglende data

rows_before = len(df)

missing_value = df.dropna(subset=['value'])

rows_after = len(missing_value)

rows_removed = rows_before - rows_after

print(f'Antall rader fjernet er: {rows_removed}')




Antall rader fjernet er: 0


# Finner gjennomsnittsverdier per dag
Vi ser at alle målinger er innefor vårt gyldighetsområde og det er verdier for hver dag, og jobber derfor videre med alle rader. I csv-filen er det to målinger per dag for air_temperature og presipitation_amount. Disse vil vi finne gjennomsnittet av, slik at vi kun har en måling per element per dag. Med pandas funksjonen groupby fjernes også automatisk kolonnene timeOffset og referenceTime, slik vi ønsker.

Framgangsmåten er å bruke groupby for å gruppere etter dato og elementId. Under kolonnen "value" finner vi gjennomsnittet over gruppen, og for kolonnen "unit" brukes første enhet. Verdiene rundes av til 3 desimaler.

In [15]:
# Gruppér og ta gjennomsnitt per dag og elementId
aggregert = df.groupby(['date', 'elementId']).agg({
    'value': 'mean',
    'unit': 'first',  # beholder enheten
}).round(3).reset_index()

aggregert.to_csv('../data/gjsnitt_data.csv')

print(aggregert)




             date                      elementId  value  unit
0      1975-01-01      mean(air_temperature P1D)  -1.35  degC
1      1975-01-01           mean(wind_speed P1D)   1.00   m/s
2      1975-01-01  sum(precipitation_amount P1D)   3.70    mm
3      1975-01-02      mean(air_temperature P1D)  -0.75  degC
4      1975-01-02           mean(wind_speed P1D)   1.00   m/s
...           ...                            ...    ...   ...
50392  2020-12-29           mean(wind_speed P1D)   5.30   m/s
50393  2020-12-29  sum(precipitation_amount P1D)   5.80    mm
50394  2020-12-30      mean(air_temperature P1D)   2.00  degC
50395  2020-12-30           mean(wind_speed P1D)   5.10   m/s
50396  2020-12-30  sum(precipitation_amount P1D)   5.75    mm

[50397 rows x 4 columns]
